In [55]:
import sys
import csv
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import AerSimulator
from qiskit.transpiler.passes import SabreSwap
from qiskit.circuit.library import QFT
import time

## Initial the Setting

In [56]:
# arch = "lattice_surgery"
# N = 10
coupling_map = []
num_qubits = N * N

if N < 2:
    print("N should be larger than 1")
    exit

## Generate Coupling Graph

In [57]:
# Draw rows
for i in range(N):
    for j in range(N - 1):
        pos = i * N + j
        coupling_map.append([pos, pos+1])
        coupling_map.append([pos+1, pos])
print(coupling_map)

[[0, 1], [1, 0], [1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3], [4, 5], [5, 4], [5, 6], [6, 5], [6, 7], [7, 6], [7, 8], [8, 7], [8, 9], [9, 8], [10, 11], [11, 10], [11, 12], [12, 11], [12, 13], [13, 12], [13, 14], [14, 13], [14, 15], [15, 14], [15, 16], [16, 15], [16, 17], [17, 16], [17, 18], [18, 17], [18, 19], [19, 18], [20, 21], [21, 20], [21, 22], [22, 21], [22, 23], [23, 22], [23, 24], [24, 23], [24, 25], [25, 24], [25, 26], [26, 25], [26, 27], [27, 26], [27, 28], [28, 27], [28, 29], [29, 28], [30, 31], [31, 30], [31, 32], [32, 31], [32, 33], [33, 32], [33, 34], [34, 33], [34, 35], [35, 34], [35, 36], [36, 35], [36, 37], [37, 36], [37, 38], [38, 37], [38, 39], [39, 38], [40, 41], [41, 40], [41, 42], [42, 41], [42, 43], [43, 42], [43, 44], [44, 43], [44, 45], [45, 44], [45, 46], [46, 45], [46, 47], [47, 46], [47, 48], [48, 47], [48, 49], [49, 48], [50, 51], [51, 50], [51, 52], [52, 51], [52, 53], [53, 52], [53, 54], [54, 53], [54, 55], [55, 54], [55, 56], [56, 55], [56, 57], [57, 

In [58]:
# Draw Straight Columns 
for i in range(N - 2):
    for j in range(N):
        pos = i * N + j
        coupling_map.append([pos, pos + 2 * N])
        coupling_map.append([pos + 2 * N, pos])
        print([pos, pos + 2 * N], [pos + 2 * N, pos])

[0, 20] [20, 0]
[1, 21] [21, 1]
[2, 22] [22, 2]
[3, 23] [23, 3]
[4, 24] [24, 4]
[5, 25] [25, 5]
[6, 26] [26, 6]
[7, 27] [27, 7]
[8, 28] [28, 8]
[9, 29] [29, 9]
[10, 30] [30, 10]
[11, 31] [31, 11]
[12, 32] [32, 12]
[13, 33] [33, 13]
[14, 34] [34, 14]
[15, 35] [35, 15]
[16, 36] [36, 16]
[17, 37] [37, 17]
[18, 38] [38, 18]
[19, 39] [39, 19]
[20, 40] [40, 20]
[21, 41] [41, 21]
[22, 42] [42, 22]
[23, 43] [43, 23]
[24, 44] [44, 24]
[25, 45] [45, 25]
[26, 46] [46, 26]
[27, 47] [47, 27]
[28, 48] [48, 28]
[29, 49] [49, 29]
[30, 50] [50, 30]
[31, 51] [51, 31]
[32, 52] [52, 32]
[33, 53] [53, 33]
[34, 54] [54, 34]
[35, 55] [55, 35]
[36, 56] [56, 36]
[37, 57] [57, 37]
[38, 58] [58, 38]
[39, 59] [59, 39]
[40, 60] [60, 40]
[41, 61] [61, 41]
[42, 62] [62, 42]
[43, 63] [63, 43]
[44, 64] [64, 44]
[45, 65] [65, 45]
[46, 66] [66, 46]
[47, 67] [67, 47]
[48, 68] [68, 48]
[49, 69] [69, 49]
[50, 70] [70, 50]
[51, 71] [71, 51]
[52, 72] [72, 52]
[53, 73] [73, 53]
[54, 74] [74, 54]
[55, 75] [75, 55]
[56, 76] [76

In [59]:
# Draw Diag1
for i in range(N - 1):
    for j in range(N):
        pos = i * N + j
        coupling_map.append([pos, pos + N])
        coupling_map.append([pos + N, pos])
        print([pos, pos + N], [pos + N, pos])

[0, 10] [10, 0]
[1, 11] [11, 1]
[2, 12] [12, 2]
[3, 13] [13, 3]
[4, 14] [14, 4]
[5, 15] [15, 5]
[6, 16] [16, 6]
[7, 17] [17, 7]
[8, 18] [18, 8]
[9, 19] [19, 9]
[10, 20] [20, 10]
[11, 21] [21, 11]
[12, 22] [22, 12]
[13, 23] [23, 13]
[14, 24] [24, 14]
[15, 25] [25, 15]
[16, 26] [26, 16]
[17, 27] [27, 17]
[18, 28] [28, 18]
[19, 29] [29, 19]
[20, 30] [30, 20]
[21, 31] [31, 21]
[22, 32] [32, 22]
[23, 33] [33, 23]
[24, 34] [34, 24]
[25, 35] [35, 25]
[26, 36] [36, 26]
[27, 37] [37, 27]
[28, 38] [38, 28]
[29, 39] [39, 29]
[30, 40] [40, 30]
[31, 41] [41, 31]
[32, 42] [42, 32]
[33, 43] [43, 33]
[34, 44] [44, 34]
[35, 45] [45, 35]
[36, 46] [46, 36]
[37, 47] [47, 37]
[38, 48] [48, 38]
[39, 49] [49, 39]
[40, 50] [50, 40]
[41, 51] [51, 41]
[42, 52] [52, 42]
[43, 53] [53, 43]
[44, 54] [54, 44]
[45, 55] [55, 45]
[46, 56] [56, 46]
[47, 57] [57, 47]
[48, 58] [58, 48]
[49, 59] [59, 49]
[50, 60] [60, 50]
[51, 61] [61, 51]
[52, 62] [62, 52]
[53, 63] [63, 53]
[54, 64] [64, 54]
[55, 65] [65, 55]
[56, 66] [66

In [60]:
# Draw Diag2
for i in range(N - 1):
    if i % 2 == 0:
        for j in range(1, N):
            pos = i * N + j
            coupling_map.append([pos, pos + N - 1])
            coupling_map.append([pos + N - 1, pos])
            print([pos, pos + N - 1], [pos + N - 1, pos])
    else:
        for j in range(N - 1):
            pos = i * N + j
            coupling_map.append([pos, pos + N + 1])
            coupling_map.append([pos + N + 1, pos])
            print([pos, pos + N + 1], [pos + N + 1, pos])


[1, 10] [10, 1]
[2, 11] [11, 2]
[3, 12] [12, 3]
[4, 13] [13, 4]
[5, 14] [14, 5]
[6, 15] [15, 6]
[7, 16] [16, 7]
[8, 17] [17, 8]
[9, 18] [18, 9]
[10, 21] [21, 10]
[11, 22] [22, 11]
[12, 23] [23, 12]
[13, 24] [24, 13]
[14, 25] [25, 14]
[15, 26] [26, 15]
[16, 27] [27, 16]
[17, 28] [28, 17]
[18, 29] [29, 18]
[21, 30] [30, 21]
[22, 31] [31, 22]
[23, 32] [32, 23]
[24, 33] [33, 24]
[25, 34] [34, 25]
[26, 35] [35, 26]
[27, 36] [36, 27]
[28, 37] [37, 28]
[29, 38] [38, 29]
[30, 41] [41, 30]
[31, 42] [42, 31]
[32, 43] [43, 32]
[33, 44] [44, 33]
[34, 45] [45, 34]
[35, 46] [46, 35]
[36, 47] [47, 36]
[37, 48] [48, 37]
[38, 49] [49, 38]
[41, 50] [50, 41]
[42, 51] [51, 42]
[43, 52] [52, 43]
[44, 53] [53, 44]
[45, 54] [54, 45]
[46, 55] [55, 46]
[47, 56] [56, 47]
[48, 57] [57, 48]
[49, 58] [58, 49]
[50, 61] [61, 50]
[51, 62] [62, 51]
[52, 63] [63, 52]
[53, 64] [64, 53]
[54, 65] [65, 54]
[55, 66] [66, 55]
[56, 67] [67, 56]
[57, 68] [68, 57]
[58, 69] [69, 58]
[61, 70] [70, 61]
[62, 71] [71, 62]
[63, 72] [

## Run Sabre

In [61]:
qc_qft = QFT(num_qubits=num_qubits, do_swaps=False, inverse=False, name='qft')
# Decompose the QFT circuit
decomposed_qft = qc_qft.decompose()
# decomposed_qft.draw(output = 'text', filename = 'decomposed.pdf', fold = 400)
sabre_circuit = transpile(decomposed_qft, backend=AerSimulator(),
                            coupling_map=coupling_map,
                            seed_transpiler=1,    # Sets random seed for the stochastic parts of the transpiler
                            optimization_level=1, # How much optimization to perform on the circuits.
                            layout_method='sabre',
                            routing_method='sabre',
                            basis_gates=['swap', 'cx', 'cp', 'u'])

### Count  Sabre Circuit

In [62]:
print(f'\nDepth: {sabre_circuit.depth()}\ngate count: {sabre_circuit.count_ops()}\n')


Depth: 981
gate count: OrderedDict([('cp', 4950), ('swap', 2365), ('u', 100)])



In [64]:
res = {'arch': arch, 'num_qubits': num_qubits, 'scale': N, 'depth': sabre_circuit.depth(), 'swap': sabre_circuit.count_ops()['swap']}
print(res)
datas.append(res)
header = list(res.keys())

{'arch': 'lattice_surgery', 'num_qubits': 100, 'scale': 10, 'depth': 981, 'swap': 2365}


NameError: name 'datas' is not defined